In [1]:
# Dependencies
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from datetime import datetime
from json import loads

In [2]:
# Setup Kafka and Spark Streaming
KAFKA_TOPIC = "social_media_topic"
BOOTSTRAP_SERVER = "localhost:9092"

ssc = StreamingContext(sc, 1) #stream each one second
ssc.checkpoint("./checkpoint")
lines = KafkaUtils.createDirectStream(ssc, [KAFKA_TOPIC],
                                      {"metadata.broker.list": BOOTSTRAP_SERVER})

In [3]:
def calculate_data(lines, window=60, sliding=60):
    def convert_timestamp(ts, socmed_type):
        result = ts
        if socmed_type == "twitter":
            result = datetime.strptime(ts, "%a %b %d %H:%M:%S %z %Y")
        elif socmed_type == "youtube":
            result = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%SZ")
        elif socmed_type == "instagram":
            result = datetime.fromtimestamp(int(result))
        elif socmed_type == "facebook":
            result = datetime.strptime(ts, "%Y-%m-%dT%H:%M:%S%z")

        result = result.replace(second=0, microsecond=0)
        result = datetime.strftime(result, "%Y-%m-%dT%H:%M:%S")
        return result
    
    def map_elements(line):
        el = loads(line[1])
        data = {}
        socmed_type = el["crawler_target"]["specific_resource_type"]
        timestamp = None
        
        if socmed_type == "twitter":
            timestamp = convert_timestamp(el["created_at"], socmed_type)
            data["user_count"] = 1
            data["stream_count"] = 1
            data["user_ids"] = [el["user_id_str"]]
        elif socmed_type == "youtube":
            timestamp = convert_timestamp(el["snippet"]["publishedAt"], socmed_type)
            data["user_count"] = 1
            data["stream_count"] = 1
            data["user_ids"] = [el["snippet"]["channelId"]]
        elif socmed_type == "instagram":
            timestamp = convert_timestamp(el["created_time"], socmed_type)
            data["user_count"] = 1
            data["stream_count"] = 1
            data["user_ids"] = [el["user"]["id"]]
        elif socmed_type == "facebook":
            timestamp = convert_timestamp(el["created_time"], socmed_type)
            data["user_count"] = 1
            data["stream_count"] = 1
            data["user_ids"] = [el["from"]["id"]]

        return ((socmed_type, timestamp), data)
    
    # Streaming Main
    process_result = lines.map(map_elements)
    
    return process_result
 
social_media = calculate_data(lines)
social_media.pprint()

ssc.start()
ssc.awaitTermination()

Py4JJavaError: An error occurred while calling o33.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/streaming/dstream.py", line 173, in takeAndPrint
    taken = rdd.take(num + 1)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/rdd.py", line 1360, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/pyspark/context.py", line 1069, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
  File "/home/bigdata/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/bigdata/spark-2.4.7-bin-h

-------------------------------------------
Time: 2022-04-25 15:04:06
-------------------------------------------
(('instagram', '2022-04-25T15:03:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['KPJWLIDHKWSMATHMKVKK']})
(('instagram', '2022-04-25T14:49:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['KPJWLIDHKWSMATHMKVKK']})

-------------------------------------------
Time: 2022-04-25 15:04:07
-------------------------------------------
(('instagram', '2022-04-25T14:44:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['KPJWLIDHKWSMATHMKVKK']})

-------------------------------------------
Time: 2022-04-25 15:04:08
-------------------------------------------
(('twitter', '2022-04-25T14:52:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['VOKVKZZLKTARCHNQPVTC']})
(('instagram', '2022-04-25T14:34:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['VOKVKZZLKTARCHNQPVTC']})

-------------------------------------------
Time: 2022-04-25 15:04:13
------------

-------------------------------------------
Time: 2022-04-25 15:04:30
-------------------------------------------
(('instagram', '2022-04-25T14:49:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['ZGPVGXHFXMPQAYKZVVDP']})
(('instagram', '2022-04-25T14:25:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['IRSDUTDHDVOUZZDGHGQX']})

-------------------------------------------
Time: 2022-04-25 15:04:31
-------------------------------------------
(('facebook', '2022-04-25T15:04:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['KZWZFELQFOZIDZMOABZV']})
(('twitter', '2022-04-25T15:04:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['GDTTXCOZMJBKDURVQIPK']})
(('youtube', '2022-04-25T15:04:00'), {'user_count': 1, 'stream_count': 1, 'user_ids': ['GDTTXCOZMJBKDURVQIPK']})

-------------------------------------------
Time: 2022-04-25 15:04:36
-------------------------------------------
(('facebook', '2022-04-25T15:04:00'), {'user_count': 1, 'stream_count': 1, 'user_ids':